In [21]:
import pandas as pd
import requests 

In [22]:
#read in a list of artist to gather information about
artist_df = pd.read_csv('music.csv')

In [41]:
#create an empty list to hold the data
artist_info_list = []
# loop through the DF to search for data
for index,row in artist_df.iterrows():
    artist_name  = row['artist']
    #build the query URL
    query_url = f'http://musicbrainz.org/ws/2/artist/?query=name:{artist_name}&inc=aliases&fmt=json'
    response = requests.get(query_url).json()
    #build a blank dictionary to hold data about the artist
    #creating with empty fields so even if the data doesn't exist , the ending data is consistent
    artist_info = {
        'artist': artist_name,
        'city': '',
        'started':'',
        'ended':'',
        'desc':'',
        'id':''
    }
    #assume the first result is the correct one
    artist = response['artists'][0]
    #save the musicbrainz artist id for future use
    artist_info['id'] = artist['id']
    # using try  / except on each field to populate the template dictionary
    try:
        artist_info['city']  = artist['begin-area']['name']
    except:
        pass
    try:
        artist_info['started'] = artist['life-span']['begin']
    except:
        pass
    try:
        artist_info['ended'] = artist['life-span']['end']
    except:
        pass
    try:
        artist_info['desc'] = artist['disambiguation']
    except:
        pass
    #append to the master list
    artist_info_list.append(artist_info)

In [57]:
#create a dataframe from the list of information gathered
artist_df = pd.DataFrame(artist_info_list)

In [73]:
#Using the data we gathered, I want to know every release by each artist
#calling another API endpoint to populate a new DF using the same pattern as before
release_list = []
for index,row in artist_df.iterrows():
    #using the saved artist id from the last DF to get album release info
    release_url = f"https://musicbrainz.org/ws/2/release/?query=arid:{row['id']}&fmt=json"
    response = requests.get(release_url).json()
    releases = response['releases']
    #loop through all releases to build the release list
    for release in releases:
        #using a blanket try/catch , if any of these fields are missing I don't want it in my results
        try:
            release_dict = {
            'artist': row['artist'],
            'title' : release['title'],
            'tracks' : release['track-count'],
            'date' : release['date']
            }
            release_list.append(release_dict)
        except:
            pass
        

In [76]:
#create a dataframe for the releases
releases_df = pd.DataFrame(release_list)

In [79]:
#set the index for both dataframes to be the artist name
releases_df = releases_df.set_index('artist')
artist_df = artist_df.set_index('artist')

In [83]:
#merge / inner join the two dataframes together to build a flattened dataframe with all of the information collected, joined on the indexes
final_final_df = pd.merge(releases_df,artist_df,left_index=True, right_index=True)

In [84]:
#save to a csv for importing into a database
final_final_df.to_csv('artist_info.csv')

In [ ]:
#stay tuned for part 2, the importing into postgres and cleaning the data in SQL